#### Import libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from pprint import pprint

#### Download data + description of data

In [176]:
df = pd.read_pickle('MADE_sample_final_011222.pickle')
df.head()

,CLIENT_ID,RETRO_DT,TARGET,000f55134facf53934419b254d274058d8d94f1237a7af8d47b3f40e888f437a6a83fd05e9fb4dac30515ddd740b5654c3d7b1b064f53699a59783ee93184730,036e1e790ff6f9461a0672ade8a70fd7a203b12891926838999f635bd1196ebdf728f5a92b6eebe3e0f8c263baa0ce81136cb2bbc9721b51b175904ca46c5b27,04e31da877871578755e676b9745affa310c376a6868a853fc5f3dfe8996dbaa61769bcae224fff029cc68ba97e7c8e476356508315e8e8fd58a90b919e5bf8d,09057f3e555f5b813f5ee380c8f1c1df831a6cf05007a909124f75391dd30fb098902dddd9e1b9c5631484ce30d4d6732ef75207930958e52dc592228410fa58,0aa9967667c60276babfcf1611af29c0fb5d445c3f66fec2f1c4d2515b9a94fd375e45e49137e9948712ce02c56a86b3b66dc60e819c083a24bab4877b2eda60,0d2ca31598f751eb5f1fae2457093fb542e1f8472bf602319f4614fa4f9879592c48888b943d3c35aaf4ad81822ad53f1d744eb6bebc47bd1f8d51a51822a5aa,0fd23397f0534cad0fb85ce4a48ec423e50ec597b70b66447fdad6695096dbda4f2ddc10746aab9b697387aa6c04de1aff6aa70196f132e9b51154eff857e899,...,b21180b5b5a42dbe7fa7615f95fcba885c20f0056f09d4a71dd8ec4b7a68a7b54af6340cdf8ec5006273872dbb3138ed64811ef8a81a965482bc32339f58d4ed,b3c0ef119fea1d559c5f57108af1e15c7856c03d1a37eadc66ea5cc5e1c59d258489136bed7ade7a6cf7f94d354bda76a2ed08c8dbeea1744137090bd6efa765,cf32d63dc7331321a191273099b6926e0779140433f5bd0d98974ae2344fdcf922822ebb67dd55a43a2e5d6ea9a204a556006012ac53cfce9da9780db7620d59,d07c71360870cebc65faf350e0496e799608e1ae6d107ce90e620c8051cb4910d97880185195c3690b7c8b2930e10b345fc7ed33b1bcdd92ca6eabd761cdfbcb,d1f037027b080c90d1d1de622e5407ac505df55888940d5581d2cb39ab0f73ffd1b076e65a1e2994e43ec3d6436f9d1137119d6cf139cc0d3649498ec484da39,d47be1154b5742dae83fe5c1035390c82cf3798ea7da18a2f5e503135727fd61288335aef23bf72f85dd15d77e99f6505a111154e115c83ab3a984b037f54df3,e03f20932bb3d1c2ea9cc8c05d7e6fe5fb50423d25e9827a6f849bf19b0a0e30b4c8ae81689bf9160d97a498b30a5763571506c552b9bde7c10cc8d277e4d1e4,e1e88068bae32c3b5e4a645c87fa7197ec6b839224fba1617879be7a12e69356e12b2c5b6c39dbf3aba86c7d6d561a2c11bcf44f73550d878665227dc40b1a55,e628a14912374ee53d7afe882c9b8733cc49aba9734f003ad7bdd99cfb0d9a10331997d44f3367d0db1c62e64f6fd791eda4cecd218d6280d9747b6b0ebc7237,ede52319edec5da370f89f2358deac0f33330d64a40e019fc0846d5447491a9ba1503171593c7a15323ae94e4cde1b5ae2f29dcabf2b7a24eb76963b2143f925
0,a05e38874bb65b229ac14a942b0422418e473df1000cb0...,20220612,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[2022-06-09, 1]]"
1,b71999a9bf0cbc3c950b541f9b9ca7e9cb14bba0476a5f...,20220627,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c524b01ee39955a75528cbbebab3d2dc92263bde5a835d...,20220629,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63f6393fb1fc2de80416777e9cd70d887958af32933aa9...,20220615,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,600e77d16212d6760ef9e78b7e77252973a29bef5590b4...,20220629,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2905 entries, 0 to 2904
Data columns (total 50 columns):
 #   Column                                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                                            --------------  ----- 
 0   CLIENT_ID                                                                                                                         2905 non-null   object
 1   RETRO_DT                                                                                                                          2905 non-null   int64 
 2   TARGET                                                                                                                            2905 non-null   object
 3   000f55134facf53934419b254d274058d8d94f1237a7af8d47b3f40e888f437a6a83fd05e9fb4dac30515ddd740b5654c3

In [178]:
df['TARGET'].value_counts()

0    2849
1      56
Name: TARGET, dtype: int64

Структура данных следующая:

- id пользователя
- target - является ли пользователь заинтересованным в данном товаре в конкретную дату
- ретродата - конкретная дата, в которую мы бы хотели уметь предсказывать интерес пользователя
- все следующие столбцы описывают поведение пользователя в сети в предыдущие 2 недели.
    - каждый столбец соответствует определнной ссылке
    - ссылки анонимизированы 
    - ссылки разбиты на группы, группы определяются по префиксу столбца, в группах может быть разное количество ссылок
    - история представляет собой список пар - (дата посещения, количество посещений в эту дату)

#### General description of data

In [179]:
# table size
print("df shape         :", df.shape)
# number of unique users
print("unique users     :", df.CLIENT_ID.nunique())
# number of groups
groups_columns = df.columns.tolist()[3:]

df shape         : (2905, 50)
unique users     : 2905


#### Data processing

First of all lets convert lists in cells into real lists (now they are string-lists).

In [180]:
def make_normal_list(string_list):
    if isinstance(string_list, float):
        return None
    day_process = lambda x: (pd.to_datetime(x[0], format='%Y/%m/%d').date(), x[1])
    
    history = [day_process(day) for day in string_list]
    return history

In [181]:
def make_normal_list_series(series):
    return series.apply(make_normal_list)

In [182]:
df.RETRO_DT = pd.to_datetime(df.RETRO_DT, format='%Y%m%d').dt.date

In [183]:
df.RETRO_DT.head()

0    2022-06-12
1    2022-06-27
2    2022-06-29
3    2022-06-15
4    2022-06-29
Name: RETRO_DT, dtype: object

In [184]:
df[groups_columns] = df[groups_columns].apply(make_normal_list_series)

In [185]:
df.head()

,CLIENT_ID,RETRO_DT,TARGET,000f55134facf53934419b254d274058d8d94f1237a7af8d47b3f40e888f437a6a83fd05e9fb4dac30515ddd740b5654c3d7b1b064f53699a59783ee93184730,036e1e790ff6f9461a0672ade8a70fd7a203b12891926838999f635bd1196ebdf728f5a92b6eebe3e0f8c263baa0ce81136cb2bbc9721b51b175904ca46c5b27,04e31da877871578755e676b9745affa310c376a6868a853fc5f3dfe8996dbaa61769bcae224fff029cc68ba97e7c8e476356508315e8e8fd58a90b919e5bf8d,09057f3e555f5b813f5ee380c8f1c1df831a6cf05007a909124f75391dd30fb098902dddd9e1b9c5631484ce30d4d6732ef75207930958e52dc592228410fa58,0aa9967667c60276babfcf1611af29c0fb5d445c3f66fec2f1c4d2515b9a94fd375e45e49137e9948712ce02c56a86b3b66dc60e819c083a24bab4877b2eda60,0d2ca31598f751eb5f1fae2457093fb542e1f8472bf602319f4614fa4f9879592c48888b943d3c35aaf4ad81822ad53f1d744eb6bebc47bd1f8d51a51822a5aa,0fd23397f0534cad0fb85ce4a48ec423e50ec597b70b66447fdad6695096dbda4f2ddc10746aab9b697387aa6c04de1aff6aa70196f132e9b51154eff857e899,...,b21180b5b5a42dbe7fa7615f95fcba885c20f0056f09d4a71dd8ec4b7a68a7b54af6340cdf8ec5006273872dbb3138ed64811ef8a81a965482bc32339f58d4ed,b3c0ef119fea1d559c5f57108af1e15c7856c03d1a37eadc66ea5cc5e1c59d258489136bed7ade7a6cf7f94d354bda76a2ed08c8dbeea1744137090bd6efa765,cf32d63dc7331321a191273099b6926e0779140433f5bd0d98974ae2344fdcf922822ebb67dd55a43a2e5d6ea9a204a556006012ac53cfce9da9780db7620d59,d07c71360870cebc65faf350e0496e799608e1ae6d107ce90e620c8051cb4910d97880185195c3690b7c8b2930e10b345fc7ed33b1bcdd92ca6eabd761cdfbcb,d1f037027b080c90d1d1de622e5407ac505df55888940d5581d2cb39ab0f73ffd1b076e65a1e2994e43ec3d6436f9d1137119d6cf139cc0d3649498ec484da39,d47be1154b5742dae83fe5c1035390c82cf3798ea7da18a2f5e503135727fd61288335aef23bf72f85dd15d77e99f6505a111154e115c83ab3a984b037f54df3,e03f20932bb3d1c2ea9cc8c05d7e6fe5fb50423d25e9827a6f849bf19b0a0e30b4c8ae81689bf9160d97a498b30a5763571506c552b9bde7c10cc8d277e4d1e4,e1e88068bae32c3b5e4a645c87fa7197ec6b839224fba1617879be7a12e69356e12b2c5b6c39dbf3aba86c7d6d561a2c11bcf44f73550d878665227dc40b1a55,e628a14912374ee53d7afe882c9b8733cc49aba9734f003ad7bdd99cfb0d9a10331997d44f3367d0db1c62e64f6fd791eda4cecd218d6280d9747b6b0ebc7237,ede52319edec5da370f89f2358deac0f33330d64a40e019fc0846d5447491a9ba1503171593c7a15323ae94e4cde1b5ae2f29dcabf2b7a24eb76963b2143f925
0,a05e38874bb65b229ac14a942b0422418e473df1000cb0...,2022-06-12,0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"[(2022-06-09, 1)]"
1,b71999a9bf0cbc3c950b541f9b9ca7e9cb14bba0476a5f...,2022-06-27,0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,c524b01ee39955a75528cbbebab3d2dc92263bde5a835d...,2022-06-29,0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,63f6393fb1fc2de80416777e9cd70d887958af32933aa9...,2022-06-15,0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,600e77d16212d6760ef9e78b7e77252973a29bef5590b4...,2022-06-29,0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


#### Feature engineering

- User activities within two previous weeks
    - total visits of all sites (inside of domain)
    - total visits for every group
    - total visits for every link
    - number of active days within period (two weeks)
    - number of active days for every group
    - average visits per active day (for domain and groups)
    - number of visits on previous (or 2, 3) day

1. total visits of all sites (inside of domain)

In [186]:
def get_total_visits(history_list):
    all_sum = 0
    for history in history_list:
        if isinstance(history, list):
            all_sum += sum(info[1] for info in history)
    return all_sum
    

df['total_visits_domain'] = df.apply(lambda x: get_total_visits([x[col] for col in groups_columns]), axis=1)

In [187]:
df['total_visits_domain'].head()

0     1
1     5
2     3
3    13
4     1
Name: total_visits_domain, dtype: int64

4. number of active days within period (two weeks) for domain

In [188]:
def get_active_days(history_list, activity_thr=1):
    # calculate unique days with visits during history
    # activity_thr - number of days to make day active
    all_sum = 0
    for history in history_list:
        if isinstance(history, list):
            all_sum += len(set(day for (day, visits) in history if visits > activity_thr))
    return all_sum

            
df['n1_active_days_domain'] = df.apply(lambda x: get_active_days([x[col] for col in groups_columns], 1), axis=1)
df['n2_active_days_domain'] = df.apply(lambda x: get_active_days([x[col] for col in groups_columns], 2), axis=1)
df['n3_active_days_domain'] = df.apply(lambda x: get_active_days([x[col] for col in groups_columns], 3), axis=1)

In [189]:
df[['n1_active_days_domain', 'n2_active_days_domain', 'n3_active_days_domain']].head()

,n1_active_days_domain,n2_active_days_domain,n3_active_days_domain
0,0,0,0
1,2,1,0
2,1,0,0
3,2,1,1
4,0,0,0


6. average visits per active day (for domain and groups)

considering days with at least one visit

In [190]:
def get_av_visits_per_active_day(history_list, activity_thr=1):
    visits = list()
    for history in history_list:
        if isinstance(history, list):
            visits.extend([visits for (day, visits) in history if visits > activity_thr])
    return np.mean(visits)

df['average_visits_per_day_domain'] = df.apply(lambda x: get_av_visits_per_active_day([x[col] for col in groups_columns], 1), axis=1)

/Users/margaritaulitina/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/margaritaulitina/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [191]:
df['average_visits_per_day_domain'].head()

0    NaN
1    2.5
2    2.0
3    5.0
4    NaN
Name: average_visits_per_day_domain, dtype: float64

7. number of visits on previous (or 2, 3) day

In [192]:
def get_visits_K_days_ago(history_list, retrodate, lag=1):
    visits = list()
    for history in history_list:
        if isinstance(history, list):
            visits.extend(visits for (day, visits) in history if (retrodate - day).days == lag)
    return sum(visits)

df['visits_1_day_ago_domain'] = df.apply(lambda x: get_visits_K_days_ago([x[col] for col in groups_columns], x.RETRO_DT, 1), axis=1)
df['visits_2_day_ago_domain'] = df.apply(lambda x: get_visits_K_days_ago([x[col] for col in groups_columns], x.RETRO_DT, 2), axis=1)
df['visits_3_day_ago_domain'] = df.apply(lambda x: get_visits_K_days_ago([x[col] for col in groups_columns], x.RETRO_DT, 3), axis=1)

In [194]:
df[['visits_1_day_ago_domain', 'visits_2_day_ago_domain', 'visits_3_day_ago_domain']].head()

,visits_1_day_ago_domain,visits_2_day_ago_domain,visits_3_day_ago_domain
0,0,0,1
1,2,0,0
2,0,2,0
3,0,1,0
4,0,0,0


#### Save data for models

In [195]:
ignore_columns = groups_columns + ['RETRO_DT']
important_columns = [c for c in df.columns if c not in ignore_columns]
len(important_columns)

10

In [22]:
df[important_columns].to_csv('sample_processed.scv', index=False)

#### Future work

Можно визуализировать посчитанные статистики, должны получаться понятные гистограммы + возможно будут видны отличия по таргету.

-----

что еще можно попробовать

Ideas and brainstorming:

Нужно придумывать признаки, которые бы коррелировали с типичным поведением пользователя и его мотивами, настроением. Возможно следует посмотреть в сторону поведения пользователя не внутри одного домена, а кросс-доменное поведение. Например, кто-то смотрит объявления про машины, хочет купить авто, значит скоро возьмет кредит. Подсовываем ему кредитную рекламу. 

Нужно найти такие близкие домены. Возможно опять же нужно кластеризовать домены по поведениям пользователей. Необходимы данные по одним и тем же пользователям в разных доменах в течение +- одного времени. 

- кластеризация пользователей на группы (возможно есть активные, средние, пассивные, для каждой группы свои модели)
- добавить к датам еще время посещения, чтобы можно было учитывать время суток
- отсутствие интереса в начале двух недель, появление интереса к концу недель к ретродате
- среднее время между соседними посещениями (также для всех сайтов, групп и всего домена)
- количество дней между первым и последним заходом
- is retrodate the same weekday as one of active days
- is retrodate the next (1, 2, 3 days) after active day
- increasing of interest within two weeks (number of visits goes up)- 